# Parabola exploration (test changes in ED library)

Created on Nov, 15, 2017
@author: Juan Manuel Acevedo Valle

In [4]:
import datetime

from parabola_configurations import model_, comp_func
from exploration.systems.parabola import ParabolicRegion as System
from exploration.systems.parabola import Instructor
from exploration.algorithm.algorithm_2017 import Algorithm as Algorithm
from exploration.algorithm.evaluation import Evaluation
from exploration.data.PlotTools import *
from exploration.algorithm.utils.functions import generate_motor_grid

import os

class OBJECT(object):
    def __init__(self):
        pass

In [5]:
now = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_")
file_prefix = 'test/test_' + now
file_prefix = file_prefix.replace('/',os.sep)
proprio, mode = True, 'social'

f_im_key, f_sm_key, f_cons_key = 'explauto_im', 'igmm_sm','explauto_cons'

val1_name, val1_file = 'whole', '../../systems/datasets/parabola_v2_dataset.h5'
val2_name, val2_file = 'social', '../../systems/datasets/instructor_parabola_1.h5'

val1_file = val1_file.replace('/',os.sep)
val2_file = val2_file.replace('/',os.sep)

# To guarantee reproducible experiments
random_seed = 12455 # 12455   #1234
thres_slope= 1.

n_initialization, n_experiments, n_save_data, eval_step= 100, 1000, -1, 500
# n_save_data np.nan to not save, -1 to save 5 times during exploration

# Creating Agent ##
system, instructor = System(), Instructor(thresh_slope=thres_slope)

# Creating models ##
models = OBJECT()
models.f_sm = model_(f_sm_key, system)
models.f_cons = model_(f_cons_key, system)
models.f_im = model_(f_im_key, system)

evaluation_sim = Evaluation(system,
                            models.f_sm,
                            comp_func=comp_func,
                            file_prefix=file_prefix)

evaluation_sim.load_eval_dataset(val1_file, name = val1_name)
evaluation_sim.load_eval_dataset(val2_file, name = val2_name)

In [3]:
#  Creating Simulation object, running simulation and plotting experiments##
simulation = Algorithm(system,
                       models,
                       n_experiments,
                       comp_func,
                       instructor=instructor,
                       n_initialization_experiments=n_initialization,
                       random_seed=random_seed,
                       g_im_initialization_method='all',
                       n_save_data=n_save_data,
                       evaluation=evaluation_sim,
                       eval_step=eval_step,
                       sm_all_samples=False,
                       file_prefix=file_prefix)

simulation.mode = mode  # social or autonomous

# for logs
simulation.f_sm_key = f_sm_key
simulation.f_cons_key = f_cons_key
simulation.f_im_key = f_im_key

simulation.run(proprio=proprio)

SM Exploration (proprio, social), Line 1: Initializing G_SM
G_SM initialized
SM Exploration: IM initialization: All sensory result considered
G_IM initialized
SM Exploration (proprio, social), First evaluation of G_SM
Evaluating model with whole (441 samples)...
Evaluation with whole has been finished.
Evaluating model with social (8 samples)...
Evaluation with social has been finished.
Evaluations finished. Resuming exploration...
SM Exploration (proprio, social), Lines 4-22: : Main simulation running (1000 samples)...
SM Exploration (proprio, social), Line 4-22: Experiment: Saving data at samples 200 of 1000
SM Exploration (proprio, social), Line 4-22: Experiment: Saving data at samples 400 of 1000
Evaluating model with whole (441 samples)...
Evaluation with whole has been finished.
Evaluating model with social (8 samples)...
Evaluation with social has been finished.
Evaluations finished. Resuming exploration...
SM Exploration (proprio, social), Line 4-22: Experiment: Saving data at 

In [4]:
sim_data = simulation.data

evaluation_sim.model.set_sigma_explo_ratio(0.)
evaluation_sim.model.mode = 'exploit'

val_data = evaluation_sim.evaluate(save_data=True)
for key in val_data.keys():
    error_ = np.linalg.norm(val_data[key].sensor_goal.data.as_matrix() -
                            val_data[key].sensor.data.as_matrix(), axis=1)    
    print("Mean evaluation error for {} dataset is {} (max: {}, min: {})".format(key,
                                                                                 np.mean(error_),
                                                                                 np.min(error_),
                                                                                 np.max(error_)))

Evaluating model with whole (441 samples)...
Evaluation with whole has been finished.
Evaluating model with social (8 samples)...
Evaluation with social has been finished.
Mean evaluation error for whole dataset is 0.0680050299324 (max: 0.000481009345636, min: 0.660298819167)
Mean evaluation error for social dataset is 0.131925777116 (max: 0.00981890476771, min: 0.516414357451)


In [5]:
"""#  Looking at the proprioceptive model
    cons_th = system.cons_threshold

    n_motor_samples = 2000

    m1, m2 = generate_motor_grid(system, n_motor_samples)

    proprio_val = []
    for m in zip(m1.flatten(), m2.flatten()):
        system.cons_out = 0.
        system.set_action(np.array([m[0], m[1]]))
        cons_pred = simulation.models.f_cons.predict_cons(system)
        system.execute_action()
        cons_res = system.cons_out
        # print("We predicted {} but got {}.".format(cons_pred, cons_res))
        system.execute_action_unconstrained()

        if cons_pred >= cons_th and cons_res >= cons_th:
            proprio_val += [[system.sensor_out[0], system.sensor_out[1], '.k']]

        if cons_pred >= cons_th > cons_res:
            proprio_val += [[system.sensor_out[0], system.sensor_out[1], 'xr']]

        if cons_pred < cons_th and cons_res < cons_th:
            proprio_val += [[system.sensor_out[0], system.sensor_out[1], '.b']]

        if cons_pred < cons_th <= cons_res:
            proprio_val += [[system.sensor_out[0], system.sensor_out[1], 'xk']]
    '''
    simulation.params.expl_space='somato'
    model_to_eva = simulation.select_expl_model()
    evaluation_sim = Evaluation(system,
                                        model_to_eva,
                                        comp_func=comp_func,
                                        file_prefix=file_prefix + '_')
    evaluation_sim.load_eval_dataset('../../systems/datasets/parabola_v2_dataset.h5')

    val_ssm_data = evaluation_sim.evaluate(save_data=False, space=simulation.params.expl_space)
    val_ssm_data.cut_final_data()
    '''

    simulation.params.expl_space='sensor'
    #model_to_eva = simulation.select_expl_model()
    evaluation_sim = Evaluation(system,
                                simulation.models.f_sm,
                                comp_func=comp_func,
                                file_prefix=file_prefix + '_')
    evaluation_sim.load_eval_dataset('../../systems/datasets/parabola_v2_dataset.h5')

    val_sm_data = evaluation_sim.evaluate(save_data=False, space=simulation.params.expl_space)
    val_sm_data.cut_final_data()


    from parabola_results import show_results
    show_results(system, simulation, val_sm_data, val_sm_data, proprio_val, thres=thres_slope )
    """

'#  Looking at the proprioceptive model\n    cons_th = system.cons_threshold\n\n    n_motor_samples = 2000\n\n    m1, m2 = generate_motor_grid(system, n_motor_samples)\n\n    proprio_val = []\n    for m in zip(m1.flatten(), m2.flatten()):\n        system.cons_out = 0.\n        system.set_action(np.array([m[0], m[1]]))\n        cons_pred = simulation.models.f_cons.predict_cons(system)\n        system.execute_action()\n        cons_res = system.cons_out\n        # print("We predicted {} but got {}.".format(cons_pred, cons_res))\n        system.execute_action_unconstrained()\n\n        if cons_pred >= cons_th and cons_res >= cons_th:\n            proprio_val += [[system.sensor_out[0], system.sensor_out[1], \'.k\']]\n\n        if cons_pred >= cons_th > cons_res:\n            proprio_val += [[system.sensor_out[0], system.sensor_out[1], \'xr\']]\n\n        if cons_pred < cons_th and cons_res < cons_th:\n            proprio_val += [[system.sensor_out[0], system.sensor_out[1], \'.b\']]\n\n    

In [6]:
models.f_cons.save_model('test/test_save_model.txt', 'test/test_2017_11_16_17_58_44_sim_data.h5')

In [1]:
from exploration.systems.parabola import ParabolicRegion
from exploration.models.Constraints.ExplautoCons import load_model
system = ParabolicRegion()
x = load_model(system, 'test/test_save_model.txt')

In [6]:
system.set_action([3.,2.9])
x.predict_cons(system)

array([ 0.33412923])